# day50_pyspark_null_treatment

### Null  Values 
######  df.fillna(0)
######  df.fillna('unknown)

In [0]:
from pyspark.sql.functions import *

data = [
    (None, "Sreenivas K", " ", 10500.75, "2024-05-21", "NEFT", "sree.k@gmail.com"),
    (102, "Hari B", "NA", None, "2024-05-20", "RTGS", "hari_b@bank.com"),
    (103, "Raghav T", "Savings", 75000.00, "2024-05-19", "    ", "raghav123@bank.org"),
    (104, "Anu R", "Salary", 32000.00, "2024-05-22", "NEFT", "anu.r@bank.com"),
    (105, None, "Savings", 1500.00, "2024-05-18", None, "n/a"),
]

columns = ["cust_id", "cust_name", "account_type", "txn_amount", "txn_date", "txn_type", "email"]

df = spark.createDataFrame(data, columns)
df.display()

In [0]:
df.createOrReplaceTempView("df")
(spark.sql("""select coalesce(cust_id, 999) as cust_id, 
          coalesce(cust_name, 'Unknown') as cust_name, 
          coalesce(account_type, 'Unknown') as account_type, 
          coalesce(txn_amount, 0 ) as txn_amount,
          coalesce(email, 'Unknown') as email,
          coalesce(txn_date, 'Unknown') as txn_date,
          coalesce(txn_type, 'unknwn' ) as txn_amount from df""")).display()

In [0]:
df.display()

In [0]:
df.fillna(999).fillna('unknwn').display()

# df.fillna(999) ==> this will all numeric columns which null value will be updated with default value
# df.fillna('999') ==> this will all string columns which null value will be updated with default value



In [0]:
df.fillna('999').display()

In [0]:
df.display()

In [0]:
df.fillna({'cust_id': 999, 'cust_name': 'Default'}).display() # this syntax allows you to add specific null treatment for specific column

#-----OR------------
# df.fillna({'cust_id': 999}).fillna({'cust_name': 'Default'}).display()
df.fillna(999).fillna('unknwn').display()

In [0]:
from pyspark.sql.functions import col, lit, when, regexp_replace, upper,lower, instr,substring, concat, concat_ws, expr, trim


data = [
    (None, "Sreenivas K", " ", 10500.75, "2024-05-21", "NEFT", "NULL"),
    (102, "Hari B", "NA", None, "2024-05-20", "RTGS", "     "),
    (103, "Raghav T", "Savings", 75000.00, "2024-05-19", "    ", "null"),
    (104, "Anu R", "Salary", 32000.00, "2024-05-22", "NEFT", None),
    (105, None, "Savings", 1500.00, "2024-05-18", None, "n/a"),
    (106, None, "Savings", 1500.00, "2024-05-18", None, "test@gmail.com")
]

columns = ["cust_id", "cust_name", "account_type", "txn_amount", "txn_date", "txn_type", "email"]

df = spark.createDataFrame(data, columns)
df.display()

In [0]:

(df.withColumn('email', expr("""case when upper(trim(email)) in  
                            ('NULL', 'N/A', 'NA','') then null else email end""")).fillna({'email': 'Unknown1'}).display())
    # .withColumn('account_type', expr("""case when upper(trim(account_type)) in  ('NULL', 'N/A', 'NA','') then null else account_type end"""))).fillna({'email': 'Unknown', 'account_type': 'savings'}).display()

#--------------------OR-------------------------

(df.withColumn('email', expr("""case when upper(trim(email)) in  
                            ('NULL', 'N/A', 'NA','') then null else email end"""))
    .withColumn('account_type', expr("""case when upper(trim(account_type)) in  
                            ('NULL', 'N/A', 'NA','') then null else account_type end"""))
    .fillna({'email': 'Unknown', 'account_type': 'savings'})
    .display()
)    


In [0]:
(
    df.withColumn(
        'cust_name',
        expr("""case 
                  when cust_name in ('NULL', 'N/A', 'NA', '') then null 
                  else cust_name 
               end""")
    )
    .fillna({'cust_name': 'Unknown'})
    .display()
)



In [0]:
df.withColumn(
    'cust_name',
    expr("""case 
              when cust_name in ('NULL', 'N/A', 'NA', 'null') then '001'
              else cust_name 
            end""")
).display()


In [0]:
(df.withColumn('email', when(trim(lower(col('email'))).isin('na','null','','n/a','none'),None).otherwise(col('email')))).fillna({'email': 'Unknown', 'account_type': 'savings'}).display()

In [0]:
# it is genrat by AI --00:44
from pyspark.sql.functions import col, sum as spark_sum

null_counts = df.select([spark_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
display(null_counts)

In [0]:
df = (df.withColumn('email', expr("""case when upper(trim(email)) in  
                            ('NULL', 'N/A', 'NA','') then null else email end"""))
    .withColumn('account_type', expr("""case when upper(trim(account_type)) in  ('NULL', 'N/A', 'NA','') then null else account_type end"""))).fillna({'email': 'Unknown', 'account_type': 'savings', 'cust_name': 'Unknown', 'txn_amount': 0.0, 'txn_date': 'Unknown', 'txn_type': 'Unknown', 'cust_id': 999})

In [0]:
df.display()

In [0]:
from pyspark.sql.functions import expr, col, coalesce, lit, to_date

# 1) Normalize fake text-null values to real nulls
df = (
    df.withColumn(
        "email",
        expr("""case when email is null or upper(trim(email)) in ('NULL','N/A','NA','')
                  then null else email end""")
    )
    .withColumn(
        "account_type",
        expr("""case when account_type is null or upper(trim(account_type)) in ('NULL','N/A','NA','')
                  then null else account_type end""")
    )
    .withColumn(
        "cust_name",
        expr("""case when cust_name is null or upper(trim(cust_name)) in ('NULL','N/A','NA','')
                  then null else cust_name end""")
    )
    .withColumn(
        "txn_type",
        expr("""case when txn_type is null or upper(trim(txn_type)) in ('NULL','N/A','NA','')
                  then null else txn_type end""")
    )
)

# 2) Fill remaining nulls with proper defaults (match types!)
#    - If txn_date is DATE type, use a real date (not 'Unknown')
#    - If txn_date is STRING type, you can fill with 'Unknown' instead
#    (Uncomment ONE of the two blocks below depending on your column type.)

# ---- If txn_date is DATE (most common) ----
df = df.withColumn("txn_date", coalesce(col("txn_date"), to_date(lit("1900-01-01"))))
df = df.fillna({
    "email": "Unknown",
    "account_type": "savings",
    "cust_name": "Unknown",
    "txn_amount": 0.0,
    "txn_type": "Unknown",
    "cust_id": 999
})

# ---- If txn_date is STRING (use this instead of the 2 lines above) ----
# df = df.fillna({
#     "email": "Unknown",
#     "account_type": "savings",
#     "cust_name": "Unknown",
#     "txn_amount": 0.0,
#     "txn_date": "Unknown",
#     "txn_type": "Unknown",
#     "cust_id": 999
# })

df.display()
